__데이터과학과소셜데이터분석 과제2 미국달러분석__
===================================

경영학과 2017312717 나종진
-------------------------

## 필요한 라이브러리 import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet


## 데이터 가져오기

### 설명 적기
ExchangeRates는 환율 데이터로 1999년 4월부터 2021년 6월까지 월별 데이터로 원/미국달러 환율 데이터임.

CurrencyFutures은 통화선물데이터로 1999년 4월부터 2021년 6월까지 월별 데이터로 미국달러선물의 계약수, 계약금액, 미체결약정으로 구성됨.

In [2]:
ExchangeRates = pd.read_csv('./AmericanExchangeRates.csv', index_col=0, na_values=0 )
CurrencyFutures = pd.read_csv('./AmericanCurrencyFutures.csv', index_col=0, na_values=0, )
ExchangeRates = ExchangeRates.fillna(0)
CurrencyFutures = CurrencyFutures.fillna(0)
print(ExchangeRates)
print(CurrencyFutures)


          원/미국달러(매매기준율)
날짜                     
1999. 04         1176.4
1999. 05         1186.3
1999. 06         1155.9
1999. 07         1206.9
1999. 08         1184.9
...                 ...
2021. 02         1108.4
2021. 03         1133.5
2021. 04         1107.4
2021. 05         1116.0
2021. 06         1130.0

[267 rows x 1 columns]
              계약수       계약금액  미결제약정수량
날짜                                   
1999. 04     1442      85401      150
1999. 05     7448     446132     2097
1999. 06    15110     882245     5161
1999. 07    35681    2125860    10311
1999. 08    29055    1741767    11416
...           ...        ...      ...
2021. 02  7751130   86311877   927596
2021. 03  8905739  100882055   878377
2021. 04  7981906   89268875   895082
2021. 05  7432911   83604348   878297
2021. 06  8025470   90161651   817949

[267 rows x 3 columns]


## 데이터 설정

앞의 파일과 같은 방식으로 변경

### 시간데이터로 변경


In [3]:
ExchangeRates.index = pd.to_datetime(ExchangeRates.index, format="%Y. %m") # dtype = 'datetime64[ns]'로
ExchangeRates.index = ExchangeRates.index.to_period("M") # dtype='period[M]'으로 
CurrencyFutures.index = pd.to_datetime(CurrencyFutures.index, format="%Y. %m") # dtype = 'datetime64[ns]'로
CurrencyFutures.index = CurrencyFutures.index.to_period("M") # dtype='period[M]'으로 

### 국가별로 환율과 통화선물 데이터 병합

In [4]:
AmericanData = pd.merge(ExchangeRates['원/미국달러(매매기준율)'], CurrencyFutures, how='outer', on='날짜')
print(AmericanData)

         원/미국달러(매매기준율)      계약수       계약금액  미결제약정수량
날짜                                                 
1999-04         1176.4     1442      85401      150
1999-05         1186.3     7448     446132     2097
1999-06         1155.9    15110     882245     5161
1999-07         1206.9    35681    2125860    10311
1999-08         1184.9    29055    1741767    11416
...                ...      ...        ...      ...
2021-02         1108.4  7751130   86311877   927596
2021-03         1133.5  8905739  100882055   878377
2021-04         1107.4  7981906   89268875   895082
2021-05         1116.0  7432911   83604348   878297
2021-06         1130.0  8025470   90161651   817949

[267 rows x 4 columns]


### 분석 데이터와 Test 데이터로 구분
( ~ 20년 12월 데이터 / 21년 1월 ~ 21년 6월 21년 상반기 데이터)

In [5]:
AmericanData_Test = AmericanData['202101':]
AmericanData_Teach = AmericanData[:'202012']

## 데이터 분석

앞의 파일과 같은 방식으로 변경

### 다중회귀분석 실행

In [6]:
X = AmericanData_Teach.iloc[:,1:]
y = pd.DataFrame(AmericanData_Teach['원/미국달러(매매기준율)'].values, columns = [['Exchange']], dtype=float)
lr2 = LinearRegression()
lr2.fit(X, y)
print(lr2.coef_)
print(lr2.intercept_)
print(lr2.score(X, y))

[[-4.36149409e-06  2.43274529e-06 -1.67420870e-04]]
[1118.77323545]
0.1149015572173101


### Test를 이용해서 비교

In [7]:
print('분석 데이터\n', lr2.predict(AmericanData_Test.iloc[:,1:]))
print('실제 데이터\n', AmericanData_Test.iloc[:,:1])

분석 데이터
 [[1167.5224236 ]
 [1139.64261029]
 [1178.29260964]
 [1151.27322726]
 [1142.69747364]
 [1166.16879372]]
실제 데이터
          원/미국달러(매매기준율)
날짜                    
2021-01         1114.6
2021-02         1108.4
2021-03         1133.5
2021-04         1107.4
2021-05         1116.0
2021-06         1130.0


### 상관관계가 높은 기간 찾기 (연별)

연도별로 상관관계가 높은 기간이 언젠지 찾는 과정.

X값에 해당하는 날짜 범위를 조절해가면서 계산.

In [8]:
date = ['200001','200101','200201','200301','200401','200501','200601','200701','200801','200901','201001','201101','201201','201301','201401','201501','201601','201701','201801','201901','202001']
date_over8 = {}
date_over9 = {}
for i in date:
  s1 = 'AmericanData_%s = AmericanData["%s":"202012"]'%(i,i)
  exec(s1)
  s2 = 'AmericanData_Repeat = AmericanData_%s'%(i)
  exec(s2)
  X = AmericanData_Repeat.iloc[:,1:]
  y = pd.DataFrame(AmericanData_Repeat["원/미국달러(매매기준율)"].values, columns = [["Exchange"]], dtype=float)
  lr2 = LinearRegression()
  lr2.fit(X, y)
  if lr2.score(X, y)>0.8:
    date_over8[i] = lr2.score(X, y)
    if lr2.score(X, y)>0.9:
      date_over9[i] = lr2.score(X, y)

print('상관관계가 0.8 이상\n', date_over8)
print('상관관계가 0.9 이상\n', date_over9)

상관관계가 0.8 이상
 {'201001': 0.8334019459764497, '201101': 0.8305211554117982, '201201': 0.8498483001024416, '201301': 0.8571623645539896, '201401': 0.8565259611196381, '201501': 0.8616128255418796, '201601': 0.8726104017469026, '201701': 0.9126998661665814, '201801': 0.9428335399399947, '201901': 0.935573328909266, '202001': 0.9585595608762761}
상관관계가 0.9 이상
 {'201701': 0.9126998661665814, '201801': 0.9428335399399947, '201901': 0.935573328909266, '202001': 0.9585595608762761}


### 가장 상관관계가 높았던 2018년으로 계산(2020년이 더 상관관계가 높지만 너무 단기간의 데이터로 제외.)

위 방법과 동일한 방법으로 계산

In [9]:
AmericanData_Test = AmericanData['202101':]
AmericanData_Teach = AmericanData['201801':'202012']
X = AmericanData_Teach.iloc[:,1:]
y = pd.DataFrame(AmericanData_Teach['원/미국달러(매매기준율)'].values, columns = [['Exchange']], dtype=float)
lr2 = LinearRegression()
lr2.fit(X, y)
print('분석 데이터\n', lr2.predict(AmericanData_Test.iloc[:,1:]))
print('실제 데이터\n', AmericanData_Test.iloc[:,:1])

분석 데이터
 [[1091.17739481]
 [1122.09130949]
 [1133.08928155]
 [1123.00684789]
 [1132.33578314]
 [1121.92636897]]
실제 데이터
          원/미국달러(매매기준율)
날짜                    
2021-01         1114.6
2021-02         1108.4
2021-03         1133.5
2021-04         1107.4
2021-05         1116.0
2021-06         1130.0


### 다른 회귀분석방법

벌점회귀분석방법을 사용(Elastic Net 회귀 방법)

In [10]:
elastic_net = ElasticNet()
elastic_net.fit(X, y)
# print('Elastic_Net coef:\n ', elastic_net.coef_) #13 개의 회귀계수 출력
# print('\nElastic_Net intercept: \n', elastic_net.intercept_) #절편값 출력
# print('상관관계', elastic_net.score(X, y))
print('분석 데이터\n', elastic_net.predict(AmericanData_Test.iloc[:,1:]))
print('실제 데이터\n', AmericanData_Test.iloc[:,:1])

분석 데이터
 [1091.17739107 1122.09128116 1133.08926786 1123.00682674 1132.33575773
 1121.9263617 ]
실제 데이터
          원/미국달러(매매기준율)
날짜                    
2021-01         1114.6
2021-02         1108.4
2021-03         1133.5
2021-04         1107.4
2021-05         1116.0
2021-06         1130.0


C:\Python37\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1126.9721462961236, tolerance: 8.616728555555557
  positive)
